# 3.8.4 Bungee Dunk

---

*Modeling and Simulation in Python*

Copyright 2021 Allen Downey, (License: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/))

Revised, Mike Augspurger (2021-present)

<br>

---

In [ ]:
#@title
# Import necessary libraries
from os.path import basename, exists
from os import mkdir

def download(url,folder):
    filename = folder + basename(url)
    if not exists(folder):
        mkdir(folder)
    # fetches the file at the given url if it is not already present
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

download('https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Notebooks/'
        + 'ModSimPy_Functions/modsim.py', 'ModSimPy_Functions/')

from ModSimPy_Functions.modsim import *
import pandas as pd
import numpy as np

In the previous notebook, we simulated a bungee jump with a model that took into account gravity, air resistance, and the spring force of the bungee cord, but we ignored the weight of the cord.  Now we'll include that factor, using the paper by [Heck, Uylings, and Kędzierska](http://iopscience.iop.org/article/10.1088/0031-9120/45/1/007).

<br>

Just a word about the counter-intuitive nature of this process.  As Heck et al. argue, the increased acceleration created by the chain is a result of the more general form of the Second Law:

<br>

$$\Sigma F = \frac{dp_{obj}}{dt} = \frac{dm_{obj}}{dt} v_{obj} +m_{obj}a_{obj}$$

<br>

where $p$ is momentum (i.e. $mv$).  If you have taken calculus, you might recognize the use of the product rule here (if you haven't, no big deal!).  But what does this mean?  And how is it possible that acceleration is *not* equal to $\frac{F}{m}$?!

<br>

Consider a military space ship cruising through space with its engines off.  There is no outside force being applied to the system, so it is not accelerating, but it is moving at a quick constant velocity.  It is dropping a space mine off every couple of seconds, but it doesn't want the mine to move after its been dropped off.  In essence, the ship has to *push* the mine backwards to reduce the mine's velocity to zero, and in doing so the mine pushes the spaceship forward (this is a Third Law, equal/opposite reaction).  So by jettisoning mass and slowing that mass down, the ship is sped up.  The larger the mine, and the faster the ship, the larger the effect! This is like the falling bungee jumper leaving sections of rope behind: the momentum that the rope loses as it stops is passed onto the remaining rope and the object.  

<br>

That's some crazy physics for you, right there.  Yessirree!

## Setting up the problem

We'll make the same assumptions we did in the previous notebook, but add this new force.  Here's is a `params` dictionary that holds the parameters, including a cord mass of 75 kg:


In [ ]:
params = dict(y_attach = 80,   # m,
                 v_init = 0,     # m / s,
                 g = 9.8,        # m/s**2,
                 m_person = 75,   # kg,
                 m_cord = 75,    # kg
                 area = 1,       # m**2,
                 rho = 1.2,      # kg/m**3,
                 v_term = 60,    # m / s,
                 L = 25,         # m,
                 k = 40)         # N / m

Now here's a `make_system` function that computes both $C_d$ and $\mu$ (the ratio of the mass of cord to the mass of the falling object), and the `drag_force` function:

In [ ]:
def make_system(params):
    y_attach, v_init, g, m_person, m_cord, area, rho, v_term, L, k = params.values()
    
    # back out the coefficient of drag
    C_d = 2 * m_person * g / (rho * area * v_term**2)
    
    mu = m_cord / m_person
    init = pd.Series(dict(y=y_attach, v=v_init))
    t_end = 8

    return dict(y_attach=y_attach, v_init=v_init, g=g,
                m_person=m_person, m_cord=m_cord, area=area, rho=rho,
                v_term=v_term, L=L, k=k,mu=mu,
                C_d=C_d, init=init, t_end=t_end)

def drag_force(v, system):
    """Computes drag force in the opposite direction of `v`.
    
    v: velocity
    
    returns: drag force in N
    """
    rho, C_d, area = system['rho'], system['C_d'], system['area']

    f_drag = -np.sign(v) * rho * v**2 * C_d * area / 2
    return f_drag

Here's drag force at 20 m/s.

In [ ]:
system = make_system(params)
drag_force(20, system)

### Part 1

Now we need to add the new force.  To do this, you'll need to pay careful attention to the definitions of the variables in the paper.  Here is the equation for the contribution to acceleration made by the cord: 

$a_{cord} = \frac{\frac{1}{2}\mu v^2}{\mu(L-y) + 2L}$ 

Because the cord creates an acceleration in two ways (its mass and its stretching), you are going to need some kind of `if` clause to determine which calculation should be used.

In [ ]:
def cord_acc(y, v, system):

    

    return a_cord

# Here's the old spring force function to borrow from if you need it
def spring_force(y, system):
    """Computes the force of the bungee cord on the jumper:
    
    y: height of the jumper
    y_attach: height of the attachment point
    L: resting length of the cord
    k: spring constant of the cord
    
    returns: force in N
    """
    y_attach, L, k = system['y_attach'], system['L'], system['k']
    
    distance_fallen = y_attach - y
    if distance_fallen <= L:
        return 0
    
    extension = distance_fallen - L
    f_spring = k * extension
    return f_spring

In the cell below, we can test acceleration due to tension in the cord if we're going 20 m/s after falling 20 m.  The return value should be -3.64 $m/s^2$.

In [ ]:
y = 60
v = -20
cord_acc(y, v, system1)

### Part 2

Now add the new function to the slope function.  Here's the old slope function:

In [ ]:
# Note: this is the old slope function
# You'll need to make some small changes to it in order
# that it work with the new cord function
def slope_func(t, state, system):
    """Compute derivatives of the state.
    
    state: position, velocity
    t: time
    system: System object containing g, rho,
            C_d, area, and mass
    
    returns: derivatives of y and v
    """
    y, v = state
    mass, g = system['mass'], system['g']
    
    a_gravity = -g
    a_drag = drag_force(v, system) / mass
    a_spring = spring_force(y, system) / mass
    
    dvdt = a_gravity + a_drag + a_spring
    
    return v, dvdt

As always, let's test the slope function with the initial params.  The initial acceleration should be $-9.8 m/s^2$:

In [ ]:
slope_func(0, system['init'], system)

### Part 3 

Let's look at our results:

In [ ]:
results, details = run_solve_ivp(system, slope_func)
details.message

In [ ]:
# Plot the position as a function of time

We can use `min` to find the lowest point:

In [ ]:
min(results.y)

To see how big the effect of the cord is, compare the simulation with a heavy cord and a very light cord.  Use `copy()` and `update()` to create new set of paramters with a light cord and rerun the simulation under a different name:

In [ ]:
# Rerun the simulation with a light/ no cord



In [ ]:
# Plot both results on the same graph



In [ ]:
# Compare the minimum y value for each simulation



### Part 4

Let's compare the contributions of the different accelerations throughout the simulation.  We'll use the data from `results` to allow us to go back in and recalculate the accelerations:

In [ ]:
accel = pd.DataFrame([],index=results.index,
                     columns=["Drag","Gravity","Cord","Total","Zero"])
for t1 in results.index:
    y, v = results.loc[t1]
    drag = drag_force(v,system)/system['m_person']
    grav = -9.8
    cord = cord_acc(y,v,system)
    total = drag + grav + cord
    accel.loc[t1]= [drag,grav,cord,total,0.0]
accel.plot(title='Accleration due to different forces');

Notice that the plot of the acceleration caused by the mass of the cord (from 0 s to about 2.2 s) looks very similar to the acceleration of the block with a chain shown in the Heck paper cited above (see p. 66).  This provides a nice experimental validation of our simulation!

<br>

Now answer some questions about this plot:

✅ A. The total acceleration peaks at about 5 seconds.  What direction is the acceleration?   About how many "g's" of acceleration does the jumper experience?

✅ ✅ Answer A here.

✅  B. What force is making the largest contribution to the acceleration at the 5 s? 

✅ ✅ Answer B here.

✅ C. Explain the meaning of the plot (its changing direction and its magnitude) of the acceleration caused by drag.

✅ ✅ Answer C here.

✅ D. When is total acceleration equal to zero?  Explain when in the jump process this is true, and why it is zero at this point.

✅ ✅ Answer D here.